In [1]:
from cobra import Reaction
import escher
import escher.urls
import cobra
import cobra.test
import json
import os
import string
from IPython.display import HTML
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.parsimonious import optimize_minimal_flux
model = cobra.io.load_matlab_model('/Users/LAURENCE/Desktop/Senior Design/RECON1', 'RECON1')

cobra/io/__init__.py:17 UserWarning: cobra.io.sbml requires libsbml
cobra/io/mat.py:136 FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


In [ ]:
# Changes:
# nh4 is now also -1000, and changed glucose to -100
# Set "MMMm" and "OCOAT1m" rxn bounds to 0
#http://localhost:8888/notebooks/Desktop/Senior%20Design/FH_model_troubleshoot_systematic_v03.ipynb# Also, make sure EX_pi_e is also -1000, not sure if it is or not, but it shouldn't be limiting.
# 5/26/16 shut off SUCOASm when setting up model
# 5/26/16 Remove 5aop sink to verify
# 5/28/16 turned SUCOA1sm back on
# 6/4/16 Added DM_atp_c demand and constrained the lb to 0.9*1.07 or 0.0177 as obtained from maxing DM_atp_c via calculatefluxes
# 6/8/16 tried to fix objective -- going to max DM_atp_c
# 6/11/16 constrained glucose to -0.4293 and growth to 0.018

In [ ]:
#Changes round 2:
# (2.1) 6/14/16 -- make BILDGLCURte and BILGLCURte irreversible forward by setting lb to 0 since it's infeasible
# (2.2)6/14/16 Replace all h_c's with h_i's in the reactions ATPS4m, CYOOm2, NADH2-u10m, CYOR-u10m

In [ ]:
lb = [0]*len(model.reactions[1188:1591])
counter = 0
for i in model.reactions[1188:1591]:
    lb[counter] = i.lower_bound
    counter = counter + 1

In [ ]:
ub = [0]*len(model.reactions[1188:1591])
counter = 0
for i in model.reactions[1188:1591]:
    ub[counter] = i.upper_bound
    counter = counter + 1

In [ ]:
import csv
from itertools import izip

with open('Recon1_uptakes.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(model.reactions[1188:1591], lb, ub))

Dan -- 06/03

First, use the metabolite uptakes and growth rates from one of the NCI60 lines (back from calculate fluxes). I'm not sure that it will matter which one you use.
Be sure to constrain the growth rate both lower and upper bounds to the same value (with the 'b' argument).
Then, change the objective to DM_atp_c and maximize that - extract a value of maybe like 0.9 of that maximum and constrain the lower bound on DM_atp_c to that value.
Finally, change the objective back to growth and do pFBA to estimate the flux state. See if the TCA cycle looks more 'normal' now.
If things are looking more normal, retry the ranking analysis and see if that had an effect (probably still want to KO both FUM and FUMm).

### 5/28 Dan's directions: we need to fix the TCA cycle of the normal model.
#### Action: Figure out where PDH flux is going -- what fluxes are using accoa_m

* ACACT1m
* ACCOACrm
* ACGSm
* ASPNATm
* GLYATm
* HMGCOASim

### More systematic way of troubleshooting the FH model

#### Overview of FH case:
KO of mitochondrial FH should lead to build up of succoa. This buildup of succoa should feed into the heme synthesis cycle and cause an increase in heme synthesis pathway.

#### Problem with our model: 
We see no increase in pheme

#### Troubleshooting efforts thus far:
* Set up the model to try to turn on the heme pathway.
* KO both FUM and FUMm since the model may be trying to use both.
* Track the flow of metabolites starting from malate, through succoa, through the heme synthesis pathway

#### New troubleshooting trial and errors:
* (1) Going to try KO FUM (non-mitochondrial) in healthy state since there is 0 flux going through FUMm -- a modeling eror since TCA cycle is in the mitochondria. Result: We see a noticeable backup in the TCA cycle from FUMm to SUCD1m. However, there is no flux going through SUCOAS1m. Action: Need to find source of where succ is leaving and force it through SUCOA1sm

* (2) Going to find where else succ_m can go since it's not going into SUCOAS1m. Results: All flux from SUCD1m is going into SUCCtm. Action: Shut off SUCCtm when setting up the model and observe

* (3) Shut off SUCCtm and run model. Results: Flux is properly forced through heme synthesis when FH is KO.

#### Results after finishing (3):
Pheme is properly synthesized after FH KO. However, no pheme synthesis in normal healthy state so there is no "differential" but only an activation of pheme production.


In [2]:
#creat ethanalamine demand
#metabolite is etha_c

DM_etha_c = Reaction('DM_etha_c')
DM_etha_c.name = DM_etha_c

etha_c = model.metabolites.get_by_id('etha_c')

DM_etha_c.add_metabolites({etha_c: -1})
model.add_reaction(DM_etha_c)

#create a transporter for this

Tyr_ggnt = Reaction('Tyr_ggnt')
Tyr_ggnt.name = Tyr_ggnt

Tyr_ggn_e = model.metabolites.get_by_id('Tyr_ggn_e')
Tyr_ggn_c = model.metabolites.get_by_id('Tyr_ggn_c')

Tyr_ggnt.add_metabolites({Tyr_ggn_e: -1, Tyr_ggn_c: 1})
model.add_reaction(Tyr_ggnt)

DM_etha_c.lower_bound = -1
Tyr_ggnt.lower_bound = -1000

except_EX_names = '''DM_etha_c, EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e'''
except_EX_names_split = except_EX_names.split(', ')
print except_EX_names_split

# Changing all lower bounds to 0 except the given reactions above

for a in model.reactions[1188:1591]:
    a.lower_bound = 0

model.reactions.get_by_id('EX_yvite_e').lower_bound = 0
model.reactions.get_by_id('EX_10fthf5glu_e').lower_bound = 0
    
for b in model.reactions[1188:1591]:
    for c in range(len(except_EX_names_split)):  
        if b.id  == except_EX_names_split[c]:
            b.lower_bound = -1
            
biomass_NCI60 = Reaction('biomass_NCI60')
biomass_NCI60.name = biomass_NCI60

ala_L_c = model.metabolites.get_by_id('ala__L_c')
arg_L_c = model.metabolites.get_by_id('arg__L_c')
asn_L_c = model.metabolites.get_by_id('asn__L_c')
asp_L_c = model.metabolites.get_by_id('asp__L_c')
atp_c = model.metabolites.get_by_id('atp_c')
clpn_hs_c = model.metabolites.get_by_id('clpn_hs_c')
ctp_c = model.metabolites.get_by_id('ctp_c')
dag_hs_c = model.metabolites.get_by_id('dag_hs_c')
datp_c = model.metabolites.get_by_id('datp_c')
dctp_c = model.metabolites.get_by_id('dctp_c')
dgtp_c = model.metabolites.get_by_id('dgtp_c')
dttp_c = model.metabolites.get_by_id('dttp_c')
gln_L_c = model.metabolites.get_by_id('gln__L_c')
glu_L_c = model.metabolites.get_by_id('glu__L_c')
gly_c = model.metabolites.get_by_id('gly_c')
glygn2_c = model.metabolites.get_by_id('glygn2_c')
gtp_c = model.metabolites.get_by_id('gtp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
hdca_c = model.metabolites.get_by_id('hdca_c')
hdcea_c = model.metabolites.get_by_id('hdcea_c')
ile_L_c = model.metabolites.get_by_id('ile__L_c')
leu_L_c = model.metabolites.get_by_id('leu__L_c')
lpchol_hs_c = model.metabolites.get_by_id('lpchol_hs_c')
lys_L_c = model.metabolites.get_by_id('lys__L_c')
mag_hs_c = model.metabolites.get_by_id('mag_hs_c')
ocdca_c = model.metabolites.get_by_id('ocdca_c')
ocdcea_c = model.metabolites.get_by_id('ocdcea_c')
pa_hs_c = model.metabolites.get_by_id('pa_hs_c')
pail_hs_c = model.metabolites.get_by_id('pail_hs_c')
pchol_hs_c = model.metabolites.get_by_id('pchol_hs_c')
pe_hs_c = model.metabolites.get_by_id('pe_hs_c')
phe_L_c = model.metabolites.get_by_id('phe__L_c')
ps_hs_c = model.metabolites.get_by_id('ps_hs_c')
ser_L_c = model.metabolites.get_by_id('ser__L_c')
sphmyln_hs_c = model.metabolites.get_by_id('sphmyln_hs_c')
thr_L_c = model.metabolites.get_by_id('thr__L_c')
trp_L_c = model.metabolites.get_by_id('trp__L_c')
tyr_L_c = model.metabolites.get_by_id('tyr__L_c')
utp_c = model.metabolites.get_by_id('utp_c')
val_L_c = model.metabolites.get_by_id('val__L_c')
pro_L_m = model.metabolites.get_by_id('pro__L_m')
chsterol_r = model.metabolites.get_by_id('chsterol_r')
xolest_hs_r = model.metabolites.get_by_id('xolest_hs_r')
adp_c = model.metabolites.get_by_id('adp_c')
h_c = model.metabolites.get_by_id('h_c')
pi_c = model.metabolites.get_by_id('pi_c')

#remove ocdca and ocdcea
# added xolest value to cholesterol in biomass and remove xolest

biomass_NCI60.add_metabolites({ala_L_c: -0.587929, arg_L_c: -0.380280, asn_L_c: -0.323313, asp_L_c: -0.261396, 
                               atp_c: -35.033540, clpn_hs_c: -0.000624, ctp_c: -0.033435, dag_hs_c: -0.001032, 
                               datp_c: -0.014557, dctp_c: -0.009770, dgtp_c: -0.009748, dttp_c: -0.014546,
                               gln_L_c: -0.319051, glu_L_c: -0.387401, gly_c: -0.504294, glygn2_c: -0.034479,
                               gtp_c: -0.055967, h2o_c: -35.000000, hdca_c: -0.008293, hdcea_c: -0.003315, 
                               ile_L_c: -0.319813, leu_L_c: -0.548692, lpchol_hs_c: -0.002470, lys_L_c: -0.552717, 
                               mag_hs_c: -0.001456, pa_hs_c: -0.010645,
                               pail_hs_c: -0.005016, pchol_hs_c: -0.022878, pe_hs_c: -0.018211, phe_L_c: -0.170743,
                               ps_hs_c: -0.006808, ser_L_c: -0.385852, sphmyln_hs_c: -0.010215, thr_L_c: -0.378004, 
                               trp_L_c: -0.039847, tyr_L_c: -0.150141, utp_c: -0.063323, val_L_c: -0.385554,
                               pro_L_m: -0.237850, chsterol_r: -0.054102,  adp_c: 35.000000, 
                               h_c: 35.000000, pi_c: 35.000000})

model.add_reaction(biomass_NCI60)
my_objective = model.reactions.get_by_id('biomass_NCI60')
model.change_objective(my_objective)

['DM_etha_c', 'EX_peplys_e', 'EX_Tyr_ggn_e', 'EX_arg__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_chol_e', 'EX_cl_e', 'EX_glc__D_e', 'EX_gln__L_e', 'EX_gly_e', 'EX_h_e', 'EX_h2o_e', 'EX_ile__L_e', 'EX_k_e', 'EX_leu__L_e', 'EX_lys__L_e', 'EX_na1_e', 'EX_nh4_e', 'EX_o2_e', 'EX_phe__L_e', 'EX_pi_e', 'EX_pro__L_e', 'EX_ser__L_e', 'EX_thr__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_val__L_e']


### Adding and constraining DM_atp_c


atp+h2o->adp+pi+h


In [3]:
# Adding DM_atp_c reaction

DM_atp_c = Reaction('DM_atp_c')
DM_atp_c.name = DM_atp_c

atp_c = model.metabolites.get_by_id('atp_c')
adp_c = model.metabolites.get_by_id('adp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
h_c = model.metabolites.get_by_id('h_c')
pi_c = model.metabolites.get_by_id('pi_c')


DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1})
model.add_reaction(DM_atp_c)

# Setting lower bounds for DM_atp_c as per Dan

#model.reactions.get_by_id('DM_atp_c').lower_bound = 0.9*1.07

### Feeding the cell w fe2, o2, h, and nh4 for heme

In [4]:
model.reactions.get_by_id('EX_h_e').lower_bound = -1000
model.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model.reactions.get_by_id('EX_nh4_e').lower_bound = -1000

### Setting glucose to -100 (to -0.4293  as per calculate fluxes 6/11/11)

In [5]:
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4293

In [6]:
# constrained as per calculate fluxes 
model.reactions.get_by_id('biomass_NCI60').lower_bound = 0.0181
model.reactions.get_by_id('biomass_NCI60').upper_bound = 0.0181

### Setting EX_pi_e to not be limiting, initially lb was -1

In [7]:
model.reactions.get_by_id('EX_pi_e').lower_bound = -1000

### Set "MMMm", "OCOAT1m", rxn bounds to 0

In [ ]:
model.reactions.get_by_id('MMMm').lower_bound = 0
model.reactions.get_by_id('MMMm').upper_bound = 0

model.reactions.get_by_id('OCOAT1m').lower_bound = 0
model.reactions.get_by_id('OCOAT1m').upper_bound = 0

'''model.reactions.get_by_id('SUCOASm').lower_bound = 0
model.reactions.get_by_id('SUCOASm').upper_bound = 0'''

## (1)  KO FUM non mitochondrial

In [8]:
model.reactions.get_by_id('FUM').lower_bound = 0
model.reactions.get_by_id('FUM').upper_bound = 0

## (3) Shut off SUCCtm

In [ ]:
model.reactions.get_by_id('SUCCtm').lower_bound = 0
model.reactions.get_by_id('SUCCtm').upper_bound = 0

### Finding the "healthy" flux of FH
http://bigg.ucsd.edu/universal/reactions/FUM
##### Fumarate Hydratase named Fumarase as per Bigg (FUM)

# 6/14/16 Changes

## (2.1) Make BILGLCURte and the D one irreversible forward by setting lb to 0 (6/14/16)

In [9]:
model.reactions.get_by_id('BILGLCURte').lower_bound = 0

In [10]:
model.reactions.get_by_id('BILGLCURte').lower_bound

0

In [11]:
model.reactions.get_by_id('BILDGLCURte').lower_bound = 0
print model.reactions.get_by_id('BILGLCURte').lower_bound

0


## (2.2) Trying to add metabolite h_i to model

In [12]:
h_i = Metabolite('h_i', formula='h', name='h_i', compartment='i')
model.add_metabolites(h_i)

## (2.2) Add CYOOm2 and replace h_c's with h_i's

#### ATPS4m

In [13]:
print "initial:   " + model.reactions.get_by_id('ATPS4m').reaction
# delete reaction
model.remove_reactions('ATPS4m')

# re-add the reaction while replacing h_c with h_i
ATPS4m = Reaction('ATPS4m')
ATPS4m.name = ATPS4m

adp_m = model.metabolites.get_by_id('adp_m')
h_i = model.metabolites.get_by_id('h_i')
pi_m = model.metabolites.get_by_id('pi_m')
h2o_m = model.metabolites.get_by_id('h2o_m')
h_m = model.metabolites.get_by_id('h_m')
atp_m = model.metabolites.get_by_id('atp_m')

ATPS4m.add_metabolites({adp_m: -1, h_i: -4, pi_m: -1, h2o_m: 1, h_m: 3, atp_m: 1})
model.add_reaction(ATPS4m)

print "replaced with:   " + model.reactions.get_by_id('ATPS4m').reaction

initial:   pi_m + 4.0 h_c + adp_m --> 3.0 h_m + h2o_m + atp_m
replaced with:   pi_m + 4 h_i + adp_m --> h2o_m + 3 h_m + atp_m


cobra/core/Model.py:306 UserWarning: need to pass in a list


#### Deleting CYOOm3

In [14]:
model.remove_reactions('CYOOm3')

#### Adding CYOOm2

In [15]:
CYOOm2 = Reaction('CYOOm2')
CYOOm2.name = CYOOm2

h_m = model.metabolites.get_by_id('h_m')
o2_m = model.metabolites.get_by_id('o2_m')
focytC_m = model.metabolites.get_by_id('focytC_m')
h_i = model.metabolites.get_by_id('h_i')
h2o_m = model.metabolites.get_by_id('h2o_m')
ficytC_m = model.metabolites.get_by_id('ficytC_m')


CYOOm2.add_metabolites({h_m: -8, o2_m: -1, focytC_m: -4, h_i: 4, h2o_m: 2, ficytC_m: 4})
model.add_reaction(CYOOm2)

print "rxn:   " + model.reactions.get_by_id('CYOOm2').reaction

rxn:   8 h_m + o2_m + 4 focytC_m --> 2 h2o_m + 4 h_i + 4 ficytC_m


#### Replacing NADH2_u10m

In [16]:
print "initial:   " + model.reactions.get_by_id('NADH2_u10m').reaction
# delete reaction
model.remove_reactions('NADH2_u10m')

# re-add the reaction while replacing h_c with h_i
NADH2_u10m = Reaction('NADH2_u10m')
NADH2_u10m.name = NADH2_u10m

q10_m = model.metabolites.get_by_id('q10_m')
h_m = model.metabolites.get_by_id('h_m')
nadh_m = model.metabolites.get_by_id('nadh_m')
nad_m = model.metabolites.get_by_id('nad_m')
h_i = model.metabolites.get_by_id('h_i')
q10h2_m = model.metabolites.get_by_id('q10h2_m')



NADH2_u10m.add_metabolites({q10_m: -1, h_m: -5, nadh_m: -1, nad_m: 1, h_i: 4, q10h2_m: 1})
model.add_reaction(NADH2_u10m)

print "replaced with:   " + model.reactions.get_by_id('NADH2_u10m').reaction

initial:   nadh_m + 5.0 h_m + q10_m --> nad_m + q10h2_m + 4.0 h_c
replaced with:   nadh_m + 5 h_m + q10_m --> nad_m + q10h2_m + 4 h_i


#### Replacing CYOR_u10m

In [17]:
print "initial:   " + model.reactions.get_by_id('CYOR_u10m').reaction
# delete reaction
model.remove_reactions('CYOR_u10m')

# re-add the reaction while replacing h_c with h_i
CYOR_u10m = Reaction('CYOR_u10m')
CYOR_u10m.name = NADH2_u10m

ficytC_m = model.metabolites.get_by_id('ficytC_m')
h_m = model.metabolites.get_by_id('h_m')
q10h2_m = model.metabolites.get_by_id('q10h2_m')
q10_m = model.metabolites.get_by_id('q10_m')
focytC_m = model.metabolites.get_by_id('focytC_m')
h_i = model.metabolites.get_by_id('h_i')


CYOR_u10m.add_metabolites({ficytC_m: -2, h_m: -2, q10h2_m: -1, q10_m: 1, focytC_m: 2, h_i: 4})
model.add_reaction(CYOR_u10m)

print "replaced with:   " + model.reactions.get_by_id('CYOR_u10m').reaction

initial:   2.0 ficytC_m + 2.0 h_m + q10h2_m --> 4.0 h_c + q10_m + 2.0 focytC_m
replaced with:   2 h_m + 2 ficytC_m + q10h2_m --> q10_m + 2 focytC_m + 4 h_i


### Set DM_atp_c as objective

In [18]:
model.change_objective(model.reactions.get_by_id('DM_atp_c'))

In [19]:
model.reactions.get_by_id('DM_atp_c').objective_coefficient

1.0

In [ ]:
'''# objective troubleshoot

coeff_array = [0]*len(model.reactions)
counter = 0
for i in model.reactions:
    coeff_array[counter] = model.reactions.get_by_id(str(i)).objective_coefficient
    counter = counter + 1
    '''

In [ ]:
'''# objective troubleshoot

import csv
from itertools import izip

with open('objective_coeff.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(model.reactions, coeff_array))'''

In [20]:
p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [21]:
p_solution.f

412.21178777199134

In [22]:
p_solution.x_dict['DM_atp_c']

412.21178777199134

In [ ]:
model.reactions.get_by_id('FUMm')
p_solution.x_dict['FUMm']

In [ ]:
print "the normal flux of FH is %f in a healthy cell when optimized for growth" %(p_solution.x_dict['FUMm'])

### Adding Malate (mal__L_m) sink reaction that has the same flux as normal FH flux determined via pFBA

In [ ]:
'''mal__L_m = model.metabolites.get_by_id('mal__L_m')
mal__L_m_sink = Reaction('mal__L_m_sink')
mal__L_m_sink.name = mal__L_m_sink
mal__L_m_sink.add_metabolites({mal__L_m: -1})
model.add_reaction(mal__L_m_sink)'''

In [ ]:
'''# constraining the sink flux to that of normal FH flux
model.reactions.get_by_id("mal__L_m_sink").upper_bound = p_solution.x_dict['FUM']
model.reactions.get_by_id("mal__L_m_sink").lower_bound = p_solution.x_dict['FUM']'''

### Constraining FH flux

In [ ]:
model.reactions.get_by_id("FUMm").upper_bound = p_solution.x_dict['FUMm']
model.reactions.get_by_id("FUMm").lower_bound = p_solution.x_dict['FUMm']

In [ ]:
print model.reactions.get_by_id("FUMm").upper_bound
print model.reactions.get_by_id("FUMm").lower_bound

## Create and set  5aop_m  flux thing as objective

In [ ]:
'''fiveaop_m = model.metabolites.get_by_id('5aop_m')
fiveaop_m_sink = Reaction('fiveaop_m_sink')
fiveaop_m_sink.name = fiveaop_m_sink
fiveaop_m_sink.add_metabolites({fiveaop_m: -1})
model.add_reaction(fiveaop_m_sink)

my_objective = model.reactions.get_by_id('fiveaop_m_sink')
model.change_objective(my_objective)'''

## pFBA with 5aop and malate sink

In [ ]:
p_solution = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

In [ ]:
p_solution.x_dict['SUCOAS1m']

## Knocking out ARGSS

In [ ]:
# FH KO
model.reactions.get_by_id("FUM").upper_bound = 0
model.reactions.get_by_id("FUM").lower_bound = 0

model.reactions.get_by_id("FUMm").upper_bound = 0
model.reactions.get_by_id("FUMm").lower_bound = 0

In [ ]:
# Now calculate the pFBA
p_solution_FHKO = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)

## Getting the diff between 2 pFBA solns

In [ ]:
#p_solution_KO - p_solution_normal
p_solution_diff = {key: p_solution_FHKO.x_dict[key] - p_solution.x_dict.get(key, 0) for key in p_solution_FHKO.x_dict.keys()}

## Plotting the difference

In [ ]:
b = escher.Builder(map_json='/Users/LAURENCE/Desktop/Senior Design/RECON1.Central.json',
                   reaction_data=p_solution_diff,
                   # color and size according to the absolute value
                   reaction_styles=['color', 'size', 'abs', 'text'],
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': 'red', 'size': 4},
                                   {'type': 'mean', 'color': 'green', 'size': 20},
                                   {'type': 'max', 'color': 'blue', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook(scroll_behavior='pan')

In [ ]:
# For some reason FUMm has flux going through it in KO model

print p_solution.x_dict['FUMm']
print p_solution_FHKO.x_dict['FUMm']

print p_solution.x_dict['FUM']
print p_solution_FHKO.x_dict['FUM']

## (2) -- it appears that all flux from SUCD1m is going into SUCCtm

In [ ]:
# This indicates that flux is going through SUCD1m normally (without maxing 5aop) and has flux when FH is KO

print p_solution.x_dict['SUCD1m']
print p_solution_FHKO.x_dict['SUCD1m']

# Possible fluxes that uses succ_m:
# OCOAT1m, SSALxm (fluxes both 0), SUCCtm (fluxes both 0), SUCOASm (both fluxes SUCOASm)
# NOTE: OCAOT1m was already shut off
# ACTION: shut off SUCOASm when setting up the model

print p_solution.x_dict['OCOAT1m']
print p_solution_FHKO.x_dict['OCOAT1m']

print p_solution.x_dict['SSALxm']
print p_solution_FHKO.x_dict['SSALxm']

print p_solution.x_dict['SUCCtm']
print p_solution_FHKO.x_dict['SUCCtm']

In [ ]:
model.reactions.get_by_id('MALSO3tm')
model.reactions.get_by_id('MALSO4tm')
model.reactions.get_by_id('MALtm')
model.reactions.get_by_id('MALTSULtm')
model.reactions.get_by_id('MALSO3tm')
model.reactions.get_by_id('ME1m')
# the only p_solution_FHKO with a flux other than malate is MALtm
# seems like a relatively high flux state for a cell...
print "MALtm flux normal:   " + str(p_solution.x_dict['MALtm'])
print "MALtm flux KO:   " + str(p_solution_FHKO.x_dict['MALtm'])

In [ ]:
# Nothing is going through SUCOA1sm

model.reactions.get_by_id('SUCOAS1m')
print p_solution.x_dict['SUCOAS1m']
print p_solution_FHKO.x_dict['SUCOAS1m']

In [ ]:
# 5aop sink is the same value

#print p_solution.x_dict['fiveaop_m_sink']
#print p_solution_FHKO.x_dict['fiveaop_m_sink']

In [ ]:
print p_solution.x_dict['ALASm']
print p_solution_FHKO.x_dict['ALASm']

In [ ]:
print p_solution.x_dict['HMBS']
print p_solution_FHKO.x_dict['HMBS']

print p_solution.x_dict['5AOPtm']
print p_solution_FHKO.x_dict['5AOPtm']

print p_solution.x_dict['PPBNGS']
print p_solution_FHKO.x_dict['PPBNGS']

print p_solution.x_dict['HMBS']
print p_solution_FHKO.x_dict['HMBS']

print p_solution.x_dict['UPP3S']
print p_solution_FHKO.x_dict['UPP3S']

print p_solution.x_dict['UPPDC1']
print p_solution_FHKO.x_dict['UPPDC1']

print p_solution.x_dict['CPPPGO']
print p_solution_FHKO.x_dict['CPPPGO']

print p_solution.x_dict['PPPGOm']
print p_solution_FHKO.x_dict['PPPGOm']

In [ ]:
# This is the soln producing pheme
print p_solution.x_dict['FCLTm']
print p_solution_FHKO.x_dict['FCLTm']

## Escher map of fluxes of normal model (only FUM, non-mitochondrial, is KO)

In [23]:
b = escher.Builder(map_json='/Users/LAURENCE/Desktop/Senior Design/RECON1.Central.json',
                   reaction_data=p_solution.x_dict,
                   # color and size according to the absolute value
                   reaction_styles=['color', 'size', 'abs', 'text'],
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': 'red', 'size': 4},
                                   {'type': 'mean', 'color': 'green', 'size': 20},
                                   {'type': 'max', 'color': 'blue', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook(scroll_behavior='pan')

In [ ]:
model.reactions.get_by_id('SUCOASm').lower_bound

In [ ]:
model.reactions.get_by_id('SUCOASm').upper_bound

### Which fluxes are using accoa_m?
Potential:
* ACACT1m
* ACCOACrm
* ACGSm
* ASPNATm
* GLYATm
* HMGCOASim

In [ ]:
print "PDHm flux:   " + str(p_solution.x_dict['PDHm'])


In [ ]:
print "ACACT1m flux:   " + str(p_solution.x_dict['ACACT1m'])
print "ACCOACrm flux:   " + str(p_solution.x_dict['ACCOACrm'])
print "ACGSm flux:   " + str(p_solution.x_dict['ACGSm'])
print "ASPNATm flux:   " + str(p_solution.x_dict['ASPNATm'])
print "GLYATm flux:   " + str(p_solution.x_dict['GLYATm'])
print "HMGCOASim flux:   " + str(p_solution.x_dict['HMGCOASim'])

In [ ]:
p_solution.x_dict['ACACT10m']

In [ ]:
'''<Reaction ACACT1r_copy1 at 0x1096ea850>,
 <Reaction ACACT10m at 0x1096ea910>,
 <Reaction ACACT1r_copy2 at 0x1096ea990>,
 <Reaction ACACT1m at 0x1096eaa50>,
 <Reaction ACACT1x at 0x1096eab90>,
 <Reaction ACACT4p at 0x1096eac50>,
 <Reaction ACACT5p at 0x1096ead10>,
 <Reaction ACACT6p at 0x1096eadd0>,
 <Reaction ACACT7p at 0x1096eae90>,
 <Reaction ACACT8p at 0x1096eaf50>,
 <Reaction ACACT9p at 0x1096fb050>,
 <Reaction ACACt2 at 0x1096fb110>,
 <Reaction ACACt2m at 0x1096fb250>,
 <Reaction ACACtx at 0x1096fb310>,'''

In [ ]:
stoich = [0]*len(model.reactions)
counter = 0
for i in model.reactions:
    stoich[counter] = i.reaction
    counter = counter + 1

In [ ]:
abs_p_sol_x = [0]*len(model.reactions)
counter = 0
for i in p_solution.x:
    abs_p_sol_x[counter] = abs(i)
    counter = counter + 1

In [ ]:
import csv
from itertools import izip

with open('FH_troubleshoot_fluxes.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(model.reactions, p_solution.x, abs_p_sol_x, stoich))